In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd

In [63]:
def create_hist(src,c='rgb',hist_show=False,image_show=False):
    img0 = cv2.imread(src)
    # set blue and green channels to 0
    img = img0.copy()
    if c == 'blue':
        img[:, :, 0] = 0
        img[:, :, 1] = 0
    elif c == 'green':
        img[:, :, 0] = 0
        img[:, :, 2] = 0
    elif c == 'red':
        img[:, :, 1] = 0
        img[:, :, 2] = 0
    
    rsum = img0[:,:,0].sum()
    all_sum = img0[:,:,:].sum()
    
    blood_pixels = (((img[:,:,0] < 140) & (img[:,:,0] > 130)) & ((img[:,:,1] < 6) & (img[:,:,1] > 0)) & ((img[:,:,1] < 6) & (img[:,:,2] > 0))).sum()
    total_pixels = (((img[:,:,0] < 140) | (img[:,:,0] > 130)) & ((img[:,:,1] < 6) | (img[:,:,1] > 0)) & ((img[:,:,1] < 6) | (img[:,:,2] > 0))).sum()
#     for i in img0:
#         for j in i:
#             total_pixels += 1
#             if (j[0] == 138) and (j[1] == 3) and (j[2] == 3):
#                 blood_pixels += 1
    
    ### area of colored image
#     plt1 = plt
#     values, bins, _ = plt1.hist(img.ravel(),256,[2,256])
#     area = sum(np.diff(bins)*values)
    
    ### area of all bgr histograms
#     plt.hist(img0.ravel(),256,[2,256])
#     plt2 = plt
#     values, bins, _ = plt2.hist(img0.ravel(),256,[2,256])
#     area0 = sum(np.diff(bins)*values)
    
    if hist_show == True:
        plt.hist(img.ravel(),256,[2,256])
        plt.title("Image with "+c+" pixels")
        plt.show()
    if image_show == True:
        plt.imshow(img)
        
    plt.close()
    return [rsum,all_sum,blood_pixels,total_pixels]

In [62]:
img = cv2.imread("gore _images/2_3840.jpg")

(((img[:,:,0] < 140) & (img[:,:,0] > 130)) & ((img[:,:,1] < 6) & (img[:,:,1] > 0)) & ((img[:,:,1] < 6) & (img[:,:,2] > 0))).sum()

0

In [66]:
l = os.listdir("gore _images/")

c = []
for i in l:
    if i[0] not in c:
        c.append(i[0])

m = []
cnt = 0
for j in c:
    l2 = [i for i in l if i.split("_")[0] == j]
    l3 = []
    for i in l2:
        l3.append((i,int(i.replace(".jpg","").split("_")[1])))
    l3.sort(key = lambda tup: tup[1])
    l3 = [x[0] for x in l3]
    for i in l3:
        cnt += 1
        if cnt % 500 == 0:
            print(cnt)
        test = create_hist('gore _images/'+i,'red')
        m.append([i,test[0],test[1],test[2],test[3]])
        
m[:10]

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700


[['1_0.jpg', 8915441, 30829598, 0, 409920],
 ['1_30.jpg', 8892106, 30649674, 0, 409920],
 ['1_60.jpg', 8629252, 29688528, 0, 409920],
 ['1_90.jpg', 8299709, 28537639, 0, 409920],
 ['1_120.jpg', 8142442, 28089401, 0, 409920],
 ['1_150.jpg', 9132123, 31359605, 0, 409920],
 ['1_180.jpg', 8922855, 29232986, 0, 409920],
 ['1_210.jpg', 7900206, 26819642, 0, 409920],
 ['1_240.jpg', 7072621, 24505806, 0, 409920],
 ['1_270.jpg', 2898847, 11093909, 0, 409920]]

In [67]:
df = pd.DataFrame(m, columns = None)
df.columns = ['image','red_intensity_sum','all_intensity_sum','blood_pixels','total_pixels']
df['video'] = df['image'].map(lambda x:x.split("_")[0])
df['frame'] = df['image'].map(lambda x:x.split("_")[1]).replace(".jpg","")
df = df[['video','frame','image','red_intensity_sum','all_intensity_sum','blood_pixels','total_pixels']]
df['red_intensity_change'] = round(100 - (df.groupby(df['image'].map(lambda x: x.split("_")[0]))['red_intensity_sum'].shift(1)/df['red_intensity_sum'])*100,2)
df['all_intensity_change'] = round(100 - (df.groupby(df['image'].map(lambda x: x.split("_")[0]))['all_intensity_sum'].shift(1)/df['all_intensity_sum'])*100,2)
df['red_percent'] =  round(df['red_intensity_sum']*100/df['all_intensity_sum'],2)
df['red_rank'] = df.groupby('video')["red_percent"].rank()
max_map = df.groupby('video')["red_rank"].max()
df['frame_count'] = df['video'].map(max_map)
df['frame_rank_percent'] = round(df['red_rank']*100/df['frame_count'],2)
df['blood_pixels_percent'] = round(df['blood_pixels']*100/df['total_pixels'],2)
df.head()

,video,frame,image,red_intensity_sum,all_intensity_sum,blood_pixels,total_pixels,red_intensity_change,all_intensity_change,red_percent,red_rank,frame_count,frame_rank_percent,blood_pixels_percent
0,1,0.jpg,1_0.jpg,8915441,30829598,0,409920,NaN,NaN,28.92,105.5,299.0,35.28,0.0
1,1,30.jpg,1_30.jpg,8892106,30649674,0,409920,-0.26,-0.59,29.01,108.0,299.0,36.12,0.0
2,1,60.jpg,1_60.jpg,8629252,29688528,0,409920,-3.05,-3.24,29.07,110.0,299.0,36.79,0.0
3,1,90.jpg,1_90.jpg,8299709,28537639,0,409920,-3.97,-4.03,29.08,111.0,299.0,37.12,0.0
4,1,120.jpg,1_120.jpg,8142442,28089401,0,409920,-1.93,-1.60,28.99,107.0,299.0,35.79,0.0


In [71]:
# df2 = df[(df['red_intensity_change'] >= 0) & (df['all_intensity_change'] >= 0) & (df['red_intensity_change'] - df['all_intensity_change'] > 20)]
df2 = df[(df['blood_pixels_percent'] > 0)]
selected_images = df2['image'].to_list()
selected_images

[]